In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py
# exists and contains get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

The Fibonacci sequence, say $(F_n)_{n\in\mathbb N}$, is defined as $F_0=0$, $F_1=1$ and for all $n>1$, $F_n=F_{n-2}+F_{n-1}$; so it is 0, 1, 1, 2, 3, 5, 8, 13, 21, 34...

A generator function is the best option to generate the initial segment of the Fibonacci sequence of a given length, even though it can also be used to generate the member of the Fibonacci sequence of a given index:

In [ ]:
def fibonacci_sequence():
    yield 0
    yield 1
    previous, current = 0, 1
    while True:
        previous, current = current, previous + current
        yield current

In [ ]:
S = fibonacci_sequence()
list(next(S) for _ in range(19))

In [ ]:
from IPython.display import clear_output

S = fibonacci_sequence()
for _ in range(18):
    next(S)
    clear_output()
next(S)

In case only one or a few specific members of the Fibonacci sequence are needed, a simple function is more appropriate:

In [ ]:
def iterative_fibonacci(n):
    if n < 2:
        return n
    previous, current = 0, 1
    for _ in range(2, n + 1):
        previous, current = current, previous + current
    return current

iterative_fibonacci(18)

A naive recursive implementation is elegant, but too inefficient, as we will see: 

In [ ]:
def recursive_fibonacci(n):
    if n >= 2:
        return recursive_fibonacci(n - 2) + recursive_fibonacci(n - 1)
    return n

recursive_fibonacci(18)

Let an integer $n$ greater than 1 be given. Then a call to __recursive_fibonacci(n)__ involves:

* for all nonzero $k\leq n$, $F_{n-k+1}$ calls to __recursive_fibonacci(k)__;
* $F_{n-1}$ calls to __recursive_fibonacci(0)__.

In particular, __recursive_fibonacci(n)__ calls __recursive_fibonacci(1)__ $F_n$ times. Proof is by induction on $k\leq n$:

* __recursive_fibonacci(n)__ is called once indeed.
* __recursive_fibonacci(n)__ directly calls __recursive_fibonacci(n - 1)__ and does not call it indirectly, so calls it once indeed.
* For all $k<n$, __recursive_fibonacci(n - k)__ is directly called by __recursive_fibonacci(n - k + 1)__ and by __recursive_fibonacci(n - k + 2)__. By inductive hypothesis, the latter two are called directly or indirectly by __recursive_fibonacci(n)__ $F_k$ and $F_{k-1}$ times, respectively. Hence __recursive_fibonacci(n - k)__ is called by __recursive_fibonacci(n)__ $F_{k+1}$ times.
* __recursive_fibonacci(0)__ is directly called by __recursive_fibonacci(2)__, hence it is called by __recursive_fibonacci(n)__ $F_{n-1}$ times.

Let us illustrate this for $n=6$ with the following tracing function:

In [ ]:
def trace_recursive_fibonacci(n, depth):
    print('    ' * depth, f'Start of function call for n = {n}')
    if n >= 2:
        second_previous = trace_recursive_fibonacci(n - 2, depth + 1)
        previous = trace_recursive_fibonacci(n - 1, depth + 1)
        print('    ' * depth, f'End of function call for n = {n},',
              f'returning {second_previous + previous}'
             )
        return second_previous + previous
    print('    ' * depth, f'End of function call for n = {n},',
          f'returning {n}'
         )
    return n

trace_recursive_fibonacci(6, 0)

We can still save the recursive design by saving terms of the Fibonacci sequence as they get computed for the first time. As a result of processing the __def__ statement below, a dictionary, __fibonacci__, is created and initialised with the values of the first two members of the Fibonacci sequence. Then the function __memoise_fibonacci()__ is called, directly as __memoise_fibonacci(18)__, and indirectly as __memoise_fibonacci(18)__ executes. For each of those calls, __memoise_fibonacci()__ is given one argument only, so the second argument is set to its default value, namely, __fibonacci__, extended with a new key and associated value in case the condition of the __if__ statement in the body of __memoise_fibonacci()__ evaluates to __True__:

In [ ]:
def memoise_fibonacci(n, fibonacci = {0: 0, 1: 1}):
    if n not in fibonacci:
        fibonacci[n] = memoise_fibonacci(n - 2) + memoise_fibonacci(n - 2)
    return fibonacci[n]

memoise_fibonacci(18)

Let us illustrate the mechanism for $n=6$ with the following tracing function:

In [ ]:
def trace_memoise_fibonacci(n, depth, fibonacci = {0: 0, 1: 1}):
    print('    ' * depth, f'Start of function call for n = {n}')
    print('    ' * (depth + 1), f'fibonacci now is {fibonacci} ', end = '')

    if n not in fibonacci:
        print('compute value')
        fibonacci[n] = trace_memoise_fibonacci(n - 2, depth + 1) +\
                       trace_memoise_fibonacci(n - 1, depth + 1)
    else:
        print('retrieve value')
    print('    ' * depth, f'End of function call for n = {n}, '
                          f'returning {fibonacci[n]}'
         )
    return fibonacci[n]

trace_memoise_fibonacci(6, 0)

__memoise_fibonacci()__ illustrates the fact that when a function argument has a default value, that default value is not created at every function call, but at the time when Python processes the function's __def__ statement. This makes no difference for default values of a type such as __int__:

In [ ]:
 def f(x = 0):
    x += 1
    return x

# Create the argument 0 before calling f(), let x denote it,
# from the value denoted by x and 1 create 1,
# let x denote it.
f(0)
f(1)
f(2)
# Let x denote the 0 created when def was processed,
# from the value denoted by x and 1 create 1,
# let x denote it.
f()
f()
f()

But it makes a difference for default values of a type such as __list__:

In [ ]:
def g(x = [0]):
    x += [1]
    return x

# Create the argument [0] before calling g(), let x denote it,
# then extend it to [0, 1],
# let x denote the modified list.
g([0])
g([1])
g([2])
# Let x denote the list L created when def was processed,
# then and now equal to [0], 
# then extend it to [0, 1],
# let x denote the modified L.
g()
# Let x denote the list L created when def was processed,
# now equal to [0, 1], 
# then extend it to [0, 1, 1],
# let x denote the modified L.
g()
g()

What was good for __memoise_fibonacci()__ might not be the intended behaviour for other functions, in other contexts: in case a function $F$ is called without an argument for a parameter $p$ that in $F$'s definition, receives a default value $v$, one might want $p$ to always be assigned that default value, not the value currently denoted by $p$ and possibly modified from the original value of $v$ following previous calls to $F$. One should then opt for a different design:

In [ ]:
def h(x = None):
    if x is None:
        x = [0]
    x += [1]
    return x

# Create the argument [0] before calling h(), let x denote it,
# then extend it to [0, 1],
# let x denote the modified list.
h([0])
h([1])
h([2])
# Let x denote None,
# then create [0], let x denote it, 
# then extend it to [0, 1],
# let x denote the modified list.
h()
h()
h()